In [1]:
import matplotlib.pyplot as plt
import openai
import torch

from PIL import Image

import viper_context
from image_patch import ImagePatch


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /opt/conda did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/opt/conda/lib/python3.10/site-packages/cv2/../../lib64')}
  warn(msg)
/opt/conda/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


final text_encoder_type: bert-base-uncased


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Using cache found in /home/jupyter/.cache/torch/hub/intel-isl_MiDaS_master
Using cache found in /home/jupyter/.cache/torch/hub/intel-isl_MiDaS_master


In [2]:
def img_unormalize(img):
    mean = torch.tensor([0.485, 0.456, 0.406]).unsqueeze(1).unsqueeze(1) 
    var = torch.tensor([0.229, 0.224, 0.225]).unsqueeze(1).unsqueeze(1) 
    img = img*var + mean
    return torch.tensor(img*255, dtype=torch.uint8)

def imshow(img):
    img = img_unormalize(img)
    img = img.numpy().transpose(1, 2, 0)
    plt.imshow(img)
    plt.show()

In [3]:
api_file = 'gpt'
with open(api_file) as f:
    api_key = f.readline().splitlines()
openai.api_key = api_key[0]

query = 'Is there a coke bottle on top of the table?'
query = 'What is on top of the table on the left?'
query = 'How far is the table on the left?'


prompt = viper_context.context.format(query=query)

input_message = [
    {"role": "system", "content": "Only answer with a function starting def execute_command."},
    {"role": "user", "content": prompt},
]

response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=input_message,
    temperature=0,
    max_tokens=1000,
    top_p=1.0,
    frequency_penalty=0.0,
    presence_penalty=2.0,
    stop=["\"\"\""])

response_message = response["choices"][0]["message"]["content"]

print(response_message)

def execute_command(image) -> str:
    image_patch = ImagePatch(image)
    table_patches = image_patch.find("table")
    # Sort the tables from left to right
    table_patches.sort(key=lambda x: x.left)
    # Get the leftmost table
    left_table = table_patches[0]
    # Compute its depth
    distance = left_table.compute_depth()
    return f"The table on the left is approximately {distance} units away."


In [4]:
IMAGE_PATH = "imgs/example_image.png"
# image = cv2.imread(IMAGE_PATH)
image = Image.open(IMAGE_PATH)
# image_source, image = load_image(IMAGE_PATH)
image_patch = ImagePatch(image)
exec(response_message,globals())
out = execute_command(image)
print(out)

/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:909: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies i

The table on the left is approximately 15 units away.
